In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
FOLDERNAME ='bitirme/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

In [ ]:
!pip install schedule

In [ ]:
import json
import time
import schedule
import threading
import pandas as pd
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import re
import os

In [ ]:
afadData = []
kandilliData = []

In [ ]:
def getAfadData():
    start_date_str = "2023-08-23T00:00:00.000Z"
    try:
        array = []
        req = Request('https://deprem.afad.gov.tr/EventData/GetEventsByFilter', method='POST')
        req.add_header('Content-Type', 'application/json; charset=utf-8')

        current_date = datetime.now()
        end_date_str = current_date.strftime("%Y-%m-%dT%H:%M:%S.%fZ")

        data = json.dumps({
            "EventSearchFilterList": [
                {"FilterType": 9, "Value": end_date_str},
                {"FilterType": 8, "Value": start_date_str}
            ],
            "Skip": 0,
            "Take": 250000,  # Remove or set to a large value to retrieve all results
            "SortDescriptor": {"field": "eventDate", "dir": "desc"}
        })

        data = data.encode()
        content = urlopen(req, data=data).read()
        content = json.loads(content)

        for i in range(len(content["eventList"])):
            el = content["eventList"][i]
            magnitudeType = el["magnitudeType"]

            # Sadece "ml" değerlerini al
            if magnitudeType == "ML":
                ml_value = float(el["magnitude"])

                json_data = json.dumps({
                    "id": i + 1,
                    "date": el["eventDate"],
                    "timestamp": int(datetime.strptime(el["eventDate"], "%Y-%m-%dT%H:%M:%S").timestamp()),
                    "latitude": float(el["latitude"]),
                    "longitude": float(el["longitude"]),
                    "depthAFAD": float(el["depth"]),
                    "size": ml_value,
                    "location": el["location"],
                    "afadDetails": {
                        "id": el["id"],
                        "refId": el["refId"]
                    },
                    "attribute": magnitudeType
                }, sort_keys=False)

                array.append(json.loads(json_data))

        return array
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return []


In [ ]:
def getKandilliData():
    try:
        array = []
        data = urlopen('http://www.koeri.boun.edu.tr/scripts/sondepremler.asp').read()
        soup = BeautifulSoup(data, 'html.parser')
        data = soup.find_all('pre')
        data = str(data).strip().split('--------------')[2]

        data = data.split('\n')
        data.pop(0)
        data.pop()
        data.pop()
        for index in range(len(data)):
            element = str(data[index].rstrip())
            element = re.sub(r'\s\s\s', ' ', element)
            element = re.sub(r'\s\s\s\s', ' ', element)
            element = re.sub(r'\s\s', ' ', element)
            element = re.sub(r'\s\s', ' ', element)
            Args = element.split(' ')
            location = Args[8]+element.split(Args[8])[len(element.split(Args[8])) - 1].split('İlksel')[0].split('REVIZE')[0]

            # Sadece "ml" değerlerini al
            ml_value = float(Args[6].replace('-.-', '0'))

            json_data = json.dumps({
                "id": index+1,
                "date": Args[0]+" "+Args[1],
                "timestamp": int(datetime.strptime(Args[0]+" "+Args[1], "%Y.%m.%d %H:%M:%S").timestamp()),
                "latitude": float(Args[2]),
                "longitude": float(Args[3]),
                "depthKandilli": float(Args[4]),
                "size": ml_value,
                "location": location.strip(),
                "attribute": element.split(location)[1].split()[0]
            }, sort_keys=False)

            array.append(json.loads(json_data))
        return array
    except:
        return []

In [ ]:
def job():
    global afadData
    global kandilliData
    afadData = getAfadData()
    kandilliData = getKandilliData()

job()
schedule.every(5).minutes.do(job)

def threadFunction():
    while True:
        schedule.run_pending()
        time.sleep(1)

x = threading.Thread(target=threadFunction)
x.start()

In [ ]:
def filterByLocation(location, data):
    return list(filter(lambda i: location.upper() in i['location'], data))


def filterBySize(size, data):
    return list(filter(lambda i: float(size) <= float(i['size']['ml']), data))


def filterBySizeAndLocation(size, location, data):
    return list(filter(lambda i: float(size) <= float(i['size']['ml']) and location.upper() in i['location'], data))


def filterByTime(hour, data):
    now = datetime.strptime(datetime.now().strftime(
        "%Y.%m.%d %H:%M:%S"), '%Y.%m.%d %H:%M:%S')
    return [record for record in data if (now - datetime.strptime(record['date'], "%Y.%m.%d %H:%M:%S")) <= timedelta(hours=int(hour))]


def filterBySizeandtime(size, hour, data):
    filtered_by_time = filterByTime(hour, data)
    filtered_by_size = filterBySize(size, filtered_by_time)
    return filtered_by_size


def filterBySizeandtimeandlocation(size, hour, location, data):
    filtered_by_time = filterByTime(hour, data)
    filtered_by_size = filterBySize(size, filtered_by_time)
    filtered_by_location = filterByLocation(location, filtered_by_size)
    return filtered_by_location


#bu kısım filtreleme kısmı hocanin verdigi lokasyona vs gore bu kısmı filtreleyebilirim fonk kullanarak

In [ ]:
def saveDataToExcel(data, filename):
    df = pd.DataFrame(data)
    file_path = f'/content/drive/My Drive/{FOLDERNAME}{filename}'  # Update the file path
    directory = f'/content/drive/My Drive/{FOLDERNAME}'

    # Create the directory if it doesn't exist
    if not os.path.exists(directory):
        os.makedirs(directory)

    df.to_excel(file_path, index=False)

while True:

    # Save data to Excel files at each update
    if afadData:
        saveDataToExcel(afadData, 'afad_data.xlsx')
    if kandilliData:
        saveDataToExcel(kandilliData, 'kandilli_data.xlsx')
    time.sleep(300)  # Wait for 5 minutes and check again